# RAG with llamaindex-upstage

<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/llamaindex_rag.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this notebook, you will use the lamindex-upstage package to create a RAG engine that uses pdf files.

First, install the relevant packages.

In [4]:
# @title Install requirements
# @markdown First, create your upstage api key from upstage console. and set UPSTAGE_API_KEY environ variable.

!pip install -qU llama-index-llms-upstage llama-index-llms-openai llama-index-embeddings-upstage llama-index-readers-file
!pip install -qU llama-index-core
!pip install -qU python-dotenv pypdf


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


# Set API Key

In [6]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [6]:
# @title Setting Upstage llm and embedding
# @markdown You can check available models [here](https://developers.upstage.ai/docs/getting-started/models)

from llama_index.llms.upstage import Upstage
from llama_index.embeddings.upstage import UpstageEmbedding
from llama_index.core.settings import Settings

llm = Upstage()
embed_model = UpstageEmbedding(model="solar-embedding-1-large")

Settings.llm = llm
Settings.embed_model = embed_model

# Load document, Build the index

In this notebook, we read a document and split it into nodes.

To split the document into nodes, we use the [SentenceWindowNodeParser](https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/MetadataReplacementDemo/). `SentenceWindowNodeParser` parses documents into single sentences per node. Each node also contains a "window" with the sentences on either side of the node sentence. And the "window" will be used in the `MetadataReplacementNodePostProcessor` in the retreiver step.

For more information, see the [Metadata Replacement + Node Sentence Window](https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/MetadataReplacementDemo/).

In [7]:
# @title Load document, Build the index
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceWindowNodeParser

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# load pdf document
documents = SimpleDirectoryReader(input_files=["./data/solar_paper.pdf"]).load_data()

# split nodes
nodes = node_parser.get_nodes_from_documents(documents)

print("Total nodes: ", len(nodes), "\n")
print("Content:\n", nodes[0].get_content(), "\n")
print("Metadata:\n", nodes[0].metadata)

Total nodes:  595 

Content:
 SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective
Depth Up-Scaling
Dahyun Kim∗, Chanjun Park∗†, Sanghoon Kim∗†, Wonsung Lee∗†, Wonho Song∗
Yunsu Kim∗, Hyeonwoo Kim∗, Yungi Kim, Hyeonju Lee, Jihoo Kim
Changbae Ahn, Seonghoon Yang, Sukyung Lee, Hyunbyung Park, Gyoungjin Gim
Mikyoung Cha, Hwalsuk Lee†, Sunghun Kim†
Upstage AI, South Korea
{kdahyun, chanjun.park, limerobot, wonsung.lee, hwalsuk.lee, hunkim}@upstage.ai
Abstract
We introduce SOLAR 10.7B, a large language
model (LLM) with 10.7 billion parameters,
demonstrating superior performance in various
natural language processing (NLP) tasks.  

Metadata:
 {'window': 'SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective\nDepth Up-Scaling\nDahyun Kim∗, Chanjun Park∗†, Sanghoon Kim∗†, Wonsung Lee∗†, Wonho Song∗\nYunsu Kim∗, Hyeonwoo Kim∗, Yungi Kim, Hyeonju Lee, Jihoo Kim\nChangbae Ahn, Seonghoon Yang, Sukyung Lee, Hyunbyung Park, Gyoungjin Gim\nMikyoung Cha, Hwalsuk Lee†, S

In [8]:
# @title Build index
index = VectorStoreIndex(nodes)

# Querying with MetadataReplacementPostProcessor

Here, we use `MetadataReplacementPostProcessor` to replace the sentence in each node with it`s surrounding sentences.

In [9]:
# @title Querying with MetadataReplacementPostProcessor
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

window_response = query_engine.query("Who are authors of paper?")
print(window_response)

The authors of the paper are Sanghoon Kim, Dahyun Kim, Chanjun Park, Wonsung Lee, Wonho Song, Yunsu Kim, Hyeonwoo Kim, Yungi Kim, Jihoo Kim, Changbae Ahn, Seonghoon Yang, Sukyung Lee, Hyunbyung Park, Gyoungjin Gim, Hyeonju Lee, and Mikyoung Cha.


In [10]:
# @title Check source node`s metadata window and origineal text
print("Window:\n", window_response.source_nodes[0].node.metadata["window"])
print()
print(
    "Original Sentence:\n",
    window_response.source_nodes[0].node.metadata["original_text"],
)

Window:
 2023.  Llama 2: Open founda-
tion and fine-tuned chat models.  arXiv preprint
arXiv:2307.09288 .
 Lewis Tunstall, Edward Beeching, Nathan Lambert,
Nazneen Rajani, Kashif Rasul, Younes Belkada,
Shengyi Huang, Leandro von Werra, Clémentine
Fourrier, Nathan Habib, et al.  2023.  Zephyr: Di-
rect distillation of lm alignment.  arXiv preprint
arXiv:2310.16944 .

Original Sentence:
 Lewis Tunstall, Edward Beeching, Nathan Lambert,
Nazneen Rajani, Kashif Rasul, Younes Belkada,
Shengyi Huang, Leandro von Werra, Clémentine
Fourrier, Nathan Habib, et al. 
